<a href="https://colab.research.google.com/github/EveTLynn/Video-Generation/blob/main/%5BVideo_Generation%5D_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Text or image-to-video](https://huggingface.co/docs/diffusers/en/using-diffusers/text-img2vid)

[I2VGenXLPipeline](https://huggingface.co/docs/diffusers/v0.30.2/en/api/pipelines/i2vgenxl#diffusers.I2VGenXLPipeline)

In [1]:
!pip install diffusers pyngrok fastapi uvicorn python-multipart -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.7 MB/s eta 0:00:00


In [2]:
# Model
import torch
from PIL import Image
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_gif, load_image, export_to_video

# Fastapi
from fastapi import FastAPI, HTTPException, File, UploadFile, Form
from fastapi.responses import FileResponse, JSONResponse
from typing import Optional, List
from pydantic import BaseModel
import nest_asyncio
import uvicorn

# Others
import io
import os
import numpy as np
from pyngrok import ngrok
from google.colab import userdata

# Set up ngrok token
authtoken = userdata.get("ngrok_key")
ngrok.set_auth_token(authtoken)

# Load model and setup pipeline once during app startup
pipeline = I2VGenXLPipeline.from_pretrained("ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16")
pipeline.enable_model_cpu_offload()

# Helper function to load the image from file
def load_image_from_file(file: UploadFile) -> Image.Image:
    contents = file.file.read()
    image = Image.open(io.BytesIO(contents))
    return image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

vae/config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
app = FastAPI()

# Path to store uploaded images
UPLOAD_FOLDER = "uploads/"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

class GenerationRequest(BaseModel):
    image_filename: str
    prompt: str
    negative_prompt: Optional[str] = None
    num_frames: Optional[int] = 30
    fps: Optional[int] = 5
    seed: Optional[int] = 8888
    num_inference_steps: Optional[int] = 50
    guidance_scale: Optional[float] = 9.0

# Endpoint to upload image
@app.post("/upload-image/")
async def upload_image(file: UploadFile = File(...)):
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)

    # Save the uploaded file
    with open(file_path, "wb") as f:
        f.write(await file.read())

    return JSONResponse(content={"filename": file.filename}, status_code=200)


@app.post("/generate-video/")
async def generate_video(request: GenerationRequest):
    # Load image from the previously uploaded file
    image_path = os.path.join(UPLOAD_FOLDER, request.image_filename)

    # Ensure the image exists
    if not os.path.exists(image_path):
        return JSONResponse(content={"error": "Image file not found"}, status_code=404)

    # Load the image
    image = Image.open(image_path).convert("RGB")

    # Set generator seed
    generator = torch.manual_seed(request.seed)

    # Generate frames
    frames = pipeline(
        prompt=request.prompt,
        image=image,
        num_frames=request.num_frames,
        target_fps=request.fps,
        height=400,
        width=800,
        num_inference_steps=request.num_inference_steps,
        negative_prompt=request.negative_prompt,
        guidance_scale=request.guidance_scale,
        generator=generator
    ).frames

    # Convert frames to numpy array
    frames_np = [np.array(frame) for frame in frames]

    # # Export the result to a video
    # video_path = "generated_video.mp4"
    # export_to_video(frames_np[0], video_path)

    # return FileResponse(video_path, media_type='video/mp4')

    # Export the result to a GIF
    gif_path = "i2v.gif"
    export_to_gif(frames_np, gif_path)

    return FileResponse(gif_path, media_type='image/gif')

if __name__ == '__main__':
    # Use nest_asyncio to run Uvicorn in the notebook
    nest_asyncio.apply()

    # Create an ngrok tunnel to expose the FastAPI app
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

    # Run the FastAPI app with uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://c383-35-198-251-9.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [490]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image/ HTTP/1.1" 200 OK
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image/ HTTP/1.1" 200 OK
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image/ HTTP/1.1" 200 OK
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image/ HTTP/1.1" 200 OK
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /generate-video HTTP/1.1" 307 Temporary Redirect


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

  0%|          | 0/30 [00:00<?, ?it/s]

INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /generate-video/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 165, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-pack

In [ ]:
# test on colab
UPLOAD_FOLDER = "uploads/"
image_filename = 'cat.jpg'
image_path = 'cat.jpg'
prompt = "Cat disapears in the blizzard."
negative_prompt = "Distorted, discontinuous, Ugly, blurry, low resolution, motionless, static, disfigured, disconnected limbs, Ugly faces, incomplete arms"
generator = torch.manual_seed(8888)
num_frames=10
fps=5
num_inference_steps=1
guidance_scale=0.9

# Load image from the previously uploaded file
image_path = os.path.join(UPLOAD_FOLDER, image_filename)

# Load the image
image = Image.open(image_path).convert("RGB")

# Generate frames
frames = pipeline(
    prompt=prompt,
    image=image,
    num_frames=num_frames,
    target_fps=fps,
    height=400,
    width=800,
    num_inference_steps=num_inference_steps,
    negative_prompt=negative_prompt,
    guidance_scale=guidance_scale,
    generator=generator
).frames
# Check the type of frames
print(f"Type of frames: {type(frames)}")
print(f"Type of 1 frame: {type(frames[0])}")

frames_np = [np.array(frame) for frame in frames]
print(frames_np[0].shape)

video_path = "generated_video.mp4"
export_to_video(frames_np[0], video_path)

## test


In [1]:
!pip install pyngrok fastapi uvicorn python-multipart -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.8 MB/s eta 0:00:00


In [3]:
# Fastapi
from fastapi import FastAPI, HTTPException, File, UploadFile, Form
from fastapi.responses import FileResponse, JSONResponse
from typing import Optional, List
from pydantic import BaseModel
import nest_asyncio
import uvicorn

# Others
import io
import os
import numpy as np
from pyngrok import ngrok
from google.colab import userdata

# Set up ngrok token
authtoken = userdata.get("ngrok_key")
ngrok.set_auth_token(authtoken)

app = FastAPI()

# Path to store uploaded images
UPLOAD_FOLDER = "uploads/"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Endpoint to upload image
@app.post("/upload-image/")
async def upload_image(file: UploadFile = File(...)):
    file_path = os.path.join(UPLOAD_FOLDER, file.filename)

    # Save the uploaded file
    with open(file_path, "wb") as f:
        f.write(await file.read())

    return JSONResponse(content={"filename": file.filename}, status_code=200)

if __name__ == '__main__':
    # Use nest_asyncio to run Uvicorn in the notebook
    nest_asyncio.apply()

    # Create an ngrok tunnel to expose the FastAPI app
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

    # Run the FastAPI app with uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://4f6b-35-231-205-74.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [319]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image HTTP/1.1" 307 Temporary Redirect
INFO:     2405:4802:bcd1:4a40:5906:9e0a:ea24:7ab8:0 - "POST /upload-image/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [319]
